In [1]:
from __future__ import division, print_function
import os, time, scipy.io
import numpy as np
import rawpy
import glob
from pip._vendor.pkg_resources import null_ns_handler
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import torch
import torch.optim as optim
from torch.autograd import Variable

In [2]:
dic_y = {(0.01,0.01):0,(0.01,0.03):1, (0.01,0.06):2, 
         (0.08,0.01):3, (0.08,0.03):4,(0.08,0.06):5,
         (0.16,0.01):6,(0.16,0.03):7,(0.16,0.06):8 }

In [3]:
def pack_raw(raw):
    # pack Bayer image to 4 channels
    im = raw.raw_image_visible.astype(np.float32)
    im = np.maximum(im - 512, 0) / (16383 - 512)  # subtract the black level

    im = np.expand_dims(im, axis=2)
    img_shape = im.shape
    H = img_shape[0]
    W = img_shape[1]

    out = np.concatenate((im[0:H:2, 0:W:2, :],
                          im[0:H:2, 1:W:2, :],
                          im[1:H:2, 1:W:2, :],
                          im[1:H:2, 0:W:2, :]), axis=2)
    return out

In [4]:
def add_noise(img):
    #sigma_s = [0.0, 0.0, 0.0]  #0-0.16
    #sigma_c = [0.005, 0.005, 0.005]  #0-0.06
    
    img = img/100
    w, h, c = img.shape
    
    
    #sigma_s = np.random.uniform(0.0, 0.16, (4,))
    #sigma_c = np.random.uniform(0.0, 0.06, (4,))
    #changing to classification
    sig_s = np.random.choice([0.01,0.08,0.16])
    sig_c = np.random.choice([0.01,0.03,0.06])
    
    sigma_s = [sig_s]*4
    sigma_c = [sig_c]*4
    
    #add ns
    #sigma_s: 0-0.16
    sigma_s = np.reshape(sigma_s, (1, 1, c))
    noise_s_map = np.multiply(sigma_s, img) 
    noise_s = np.random.randn(w, h, c) * noise_s_map
    img = img + noise_s
    
    #add nc
    #sigma_c:0.01-0.06
    noise_c = np.zeros((w, h, c))
    for chn in range(4):
        noise_c [:, :, chn] = np.random.normal(0, sigma_c[chn], (w, h)) 
    img = img + noise_c
    
    
    return img, sig_s, sig_c

In [5]:
#path = "./Sony/Sony/long/00001_00_10s.ARW"
#raw = rawpy.imread(path)
#raw = pack_raw(raw)
#n_raw, sigma_s, sigma_c = add_noise(raw)

In [6]:
path_sony = "./Sony/Sony/long"

In [7]:
# "0" for training set and "2" for validation set, 1- test

## Read train data

In [8]:
j = 0
train_x = list()
train_y_s = list()
train_y_c = list()
train_y = list()
for i in glob.glob(path_sony + '/0*.ARW'):
    j+=1
    if(j>2):
        break
    print("reading: "+i)
    
    raw = rawpy.imread(i)
    raw = pack_raw(raw)
    
    raw, sigma_s, sigma_c = add_noise(raw)
    train_x.append(raw)
    train_y_s.append(sigma_s)
    train_y_c.append(sigma_c)
    train_y.append(dic_y[(sigma_s, sigma_c)])

reading: ./Sony/Sony/long/00088_00_30s.ARW
reading: ./Sony/Sony/long/00033_00_10s.ARW


In [9]:
len(train_y_c)

2

## Read val data

In [10]:
j = 0
val_x = list()
val_y_c = list()
val_y_s = list()
val_y = list()
for i in glob.glob(path_sony + '/2*.ARW'):
    j+=1
    if(j>2):
        break
    print("reading: "+i)
    
    raw = rawpy.imread(i)
    raw = pack_raw(raw)
    
    raw, sigma_s, sigma_c = add_noise(raw)
    val_x.append(raw)
    val_y_s.append(sigma_s)
    val_y_c.append(sigma_c)
    val_y.append(dic_y[(sigma_s, sigma_c)])

reading: ./Sony/Sony/long/20061_00_10s.ARW
reading: ./Sony/Sony/long/20153_00_30s.ARW


## Read test data

In [11]:
j = 0
test_x = list()
test_y_s = list()
test_y_c = list()
test_y = list()
for i in glob.glob(path_sony + '/1*.ARW'):
    j+=1
    if(j>2):
        break
    print("reading: "+i)
    
    raw = rawpy.imread(i)
    raw = pack_raw(raw)
    
    raw, sigma_s, sigma_c = add_noise(raw)
    test_x.append(raw)
    test_y_s.append(sigma_s)
    test_y_c.append(sigma_c)
    test_y.append(dic_y[(sigma_s, sigma_c)])

reading: ./Sony/Sony/long/10054_00_10s.ARW
reading: ./Sony/Sony/long/10199_00_10s.ARW


# Create dataset

In [12]:
class NoiseDataset(Dataset):
    
    def __init__(self, x_list, y_list, transform=None):
        self.x_list = x_list
        self.y_list = y_list
        self.transform = transform
    
    def __len__(self):
        return len(self.x_list)
    
    def __getitem__(self, idx):
        data = self.x_list[idx]
        label = self.y_list[idx]

        if self.transform:
            data = self.transform(data)
            #label = self.transform(np.asarray(label))
            

        #dict_data = {'data': data,'target': self.y_list[idx]}
        
        res = (data,label)
        return res
        

In [13]:
batch_size = 16

In [14]:
data_transforms = transforms.Compose([
    #transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

train_y = torch.FloatTensor(train_y).double()
val_y = torch.FloatTensor(val_y).double()
test_y = torch.FloatTensor(test_y).double()

train_loader = torch.utils.data.DataLoader(
    NoiseDataset(train_x, train_y,
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)
val_loader = torch.utils.data.DataLoader(
    NoiseDataset(val_x, val_y,
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(
    NoiseDataset(test_x, test_y,
                         transform=data_transforms),
    batch_size=batch_size, shuffle=True, num_workers=1)

In [15]:
for (i,j) in val_loader:
    print(i.shape)
    print(j.shape)
    break
print(len(train_loader))  


torch.Size([2, 4, 1424, 2128])
torch.Size([2])
1


In [16]:
train_loader.dataset.y_list

tensor([7., 1.], dtype=torch.float64)

## Model

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
nclasses = 9 # GTSRB has 43 classes

In [18]:
class NetL(nn.Module):

    def __init__(self):
        super(NetL, self).__init__()
        self.conv1 = nn.Conv2d(4, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 353 * 529, 120)  
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, nclasses)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [19]:
model = NetL()
lr = 0.01
epochs = 10
optimizer = optim.Adam(model.parameters(), lr=lr)
x1 = list()
y1 = list()
def trainL(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
def validationL():
    model.eval()
    validation_loss = 0
    correct = 0
    for data, target in val_loader:
        output = model(data)
        validation_loss += F.cross_entropy(output, target, reduction="sum").item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    validation_loss /= len(val_loader.dataset)
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        validation_loss, correct, len(val_loader.dataset),
        100. * correct / len(val_loader.dataset)))
    y1.append(validation_loss)
    
for epoch in range(1, epochs + 1):
    trainL(epoch)
    x1.append(epoch)
    validationL()
    model_file = 'model_' + str(epoch) + '.pth'
    torch.save(model.state_dict(), model_file)
    print('\nSaved model to ' + model_file + '.')
    
plt.plot(x1,y1)
plt.show()

RuntimeError: expected scalar type Double but found Float